# DSC 80: Homework 08

### Due Date: Monday, March 04, 12:00PM

## Instructions
Much like in DSC 10, this Jupyter Notebook contains the statements of the homework problems and provides code and markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding work will be developed in an accompanying `hw0X.py` file, that will be imported into the current notebook. (`X` is a homework number)

Homeworks and programming assignments will be graded in (at most) two ways:
1. The functions and classes in the accompanying python file will be tested (a la DSC 20),
2. The notebook will be graded (for graphs and free response questions).


**Do not change the function names in the `*.py` file**
- The functions in the `*.py` file are how your assignment is graded, and they are graded by their name. The dictionary at the end of the file (`GRADED FUNCTIONS`) contains the "grading list". The final function in the file allows your doctests to check that all the necessary functions exist.
- If you changed something you weren't supposed to, just use git to revert!

**Tips for working in the Notebook**:
- The notebooks serve to present you the questions and give you a place to present your results for later review.
- The notebook on *HW assignments* are not graded (only the `.py` file).
- Notebooks for PAs will serve as a final report for the assignment, and contain conclusions and answers to open ended questions that are graded.
- The notebook serves as a nice environment for 'pre-development' and experimentation before designing your function in your `.py` file.

**Tips for developing in the .py file**:
- Do not change the function names in the starter code; grading is done using these function names.
- Do not change the docstrings in the functions. These are there to tell you if your work is on the right track!
- You are encouraged to write your own additional functions to solve the HW! 
    - Developing in python usually consists of larger files, with many short functions.
    - You may write your other functions in an additional `.py` file that you import in `hw0X.py` (much like we do in the notebook).
- Always document your code!

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
import hw08_solutions as hw

In [5]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

# More on TF-IDF

## Most relevant document

**Question 1**

In this question we will learn how to use `tf-idf` to select the most relevant document given several queries. The overall idea is the following:

* You are given a query: a list of strings, Q.
* You are given a set of documents, S (corpus).
* You need to select a document in S that is most relevant to some query in Q. 


**The general algorithm:**

1. For each document you need to find `tf-idf` for each unique word in Q. Suppose your query Q is ["word0", "word1 word0", "word1 word2"]. The dataframe may look something like this:

| .    | doc0 |doc1
|------|------|-----
|word0 |tfidf1|tfidf4|
|word1 |tfidf2|tfidf5|
|word2 |tfidf3|tfidf6|

2. Then we calculate the weight of the queries relative to each document as a sum of TF-IDFs of each word from each query. The dataframe may look something like this:

| .    | doc0 |doc1
|------|------|-----
|word0 |tfidf1|tfidf4|
|word1 word0 |tfidf1 + tfidf2|tfidf4 + tfidf5|
|word1 word2 |tfidf2 + tfidf3|tfidf5 + tfidf6|

3. Finally, you will select the document with the max value of the sum, for example:

`
word0            doc0
word1 word0      doc0 
word1 word2      doc1
`

Write a function `most_relevant(query, corpus)` that takes in a query (as a list of strings) and a corpus and returns
a series, indexed by the strings from Q; values are position of the document in the corpus that corresponds to the maximum `tf_idf` for a particular string in a query. 

You can reuse your code from hw07 as you see fit. 


In [488]:
fp = os.path.join('data', 'corpus.txt')
corpus = pd.read_csv(fp, header=None, squeeze=True)
query = ["relevance", "text relevance", "text relevance document", "text relevance document search"]


## Testing TF-IDF understanding

**Question 2**

In this question you will answer a few multiple choice questions about the `TF-IDF` approach.

A. Based on the answer from problem 1 you should see that the most relevant document for a "text relevance" query is the 0th document. Why?

1. Because it is the first document in the corpus.
2. Because it is the first document in the corpus that has "text relevance" in it.
3. Because "text relevance" appears in the 0th document most number of times. 


B.  Based on the answer from problem 1 you should see that the most relevant document for a "text relevance document search" query is the 0th document but the words "document" and "search" never appeared in this document. What happened?

1. I think I have a bug, better fix it now. 
2. The td-idf value of one of the words in the query was large enough to keep the total high.


C. Note that none of the documents contain all words from "text relevance document search" query. Document 2 contains 3 out of 4 words. If you add word "text" to the second document, what change (if any?) will you see?

1. 0 (i.e. document 0 still has the highest sum)
2. 1 (i.e. document 1)
3. 2 (i.e. document 2)
4. 3 (i.e. document 3)

D. If you add word "relevance" in the 3rd document, what output do expect to see for "relevance" query?

1. 0 (i.e. document 0), because there is equal number of the word "relevance" in 0th and 2nd document, to break a tie we chose the first max occurrence.  
2. 0 (i.e. document 0), because the word "relevance" occurs in all documents, therefore tf-idf score is 0 for all documents, to break a tie we chose the first max occurrence. 
3. 1 (i.e. document 1), because "relevance" occurs only ones, as well as in document 3. To break a tie we chose the first max occurrence.  

E. (Unrelated to question 1) Suppose you have a set of small and large documents. You calculated the `tf-idf` values for both sets. Then you decided to check how often the words chosen are either extremely common (e.g. 'the') or distinct in your sets. What statement is correct?

1. Small documents are less likely to have common and rear words.
2. Large documents are less likely to have common and rear words.
3. The size of the documents does not matter, the likelihood is approximately the same. 

Write a function called `question2()` that does not take any parameters and returns a list with your answers. 


### Does the effectiveness of TF-IDF depend on the length of the documents?

**Question 3**

One way to attempt to answer this question is to ask: if the documents in a corpus are small, then are most TF-IDF single word summaries either very common or very rare words?

1. Define a word to be very rare if it appears (strictly) fewer than 5 times.
2. Define a word to be very common, if it is one of the top 20 words in the corpus.

To attempt to answer this, create a function `pct_rare_or_common` that takes in an iterator of documents and outputs the percentage of documents whose "best summary" (according to TF-IDF) is either a very rare or very common word.

Try your function on three datasets from the course (`ira.csv` tweets, amazon `reviews.txt`, and the state of the union addresses -- all found in the course repository).

*Note:* You should use `sklearn`'s `TfidfVectorizer` and `CountVectorizer` for this problem; if you choose to use something else, you should split the text into words using the same pattern (from sklearn: "words of 2 or more alphanumeric characters -- punctuation is completely ignored and always treated as a token separator").

# Training Models

## Examining underfitting and overfitting

Train and test a model over the same data and examine both under- and overfitting.

**Question 4**

In this problem, you will learn about how a simple model *underfits* the data; in other words, it is not expressive enough to properly fit the training/test data. You will also learn how a model that is too complex *overfits* the data, i.e. it learns the noise in the data. 

We will evaluate these two instances both qualitatively using plots and quantitatively using the mean squared error (MSE) on a validation set. The higher the MSE, the less likely the model properly generalizes training data. 

The following imports are all the methods you are expected to use for this question. We will create a new machine learning algorithm using the combination of the machine learning algorithm `LinearRegression` with the feature generation algorithm `PolynomialFeatures` to create a model of arbitrary complexity. We will combine these two algorithms using `Pipeline`.

Notes:

1. If you need to learn about/review polynomial features you can refer to: https://www.textbook.ds100.org/ch/14/feature_polynomial.html

2. For `Pipeline` objects, see the examples in lecture. A good overview is also found here: https://www.kaggle.com/baghern/a-deep-dive-into-sklearn-pipelines


### Overall idea of what needs to be done

1. First we will create a population distribution (true distribution), the one that we would like to learn.
The function `true_distribution(X)` takes a floating point number from `[0,1)` and returns a number between `[-1, 1]`.

2. Then we will create a sample (training data) from the same distribution and add some noise to it. Since we should always ensure the random number generator provides the same sequence of random numbers, we will seed it: `np.random.seed(0)`

3. Now we want to learn three different models: one that underfits the data, one that fits the data properly and one that overfits the data. The way you will do it is by building a pipeline using `PolynomialFeatures` and `LinearRegression`. You need to find 3 unique values for the degree of the polynomial features for each of the 3 different fits. 

4. In order to evaluate your predictions use the `cross_val_score` method with `cv = 10` and `scoring="neg_mean_squared_error"`

5. Plot the representation of the true curve over uniform distribution of X (`X_true`). Include your noisy samples and your learned model which is fitted over the same distribution of X as true curve. Add the degree of the polynomial and mse error to the title of each image. 



Now put everything in one function `degrees_and_mse()`, which returns a dictionary with three key-value pairs, where each key is the degree of the polynomial features (one for underfitting, one for the proper fit and the third one for the overfitting). The values are the corresponding mse errors for each model. (Note that you will use crossvalidation, which gives you multiple scores for each model, summarize to a single score).

In [389]:
import numpy as np
import matplotlib.pyplot as plt

# This is used to execute a sequence of tasks in order
from sklearn.pipeline import Pipeline

# This will be our machine learning algorithm that builds a model
from sklearn.linear_model import LinearRegression

# This is used in conjunction with linear regression to create a model of arbitary complexity
from sklearn.preprocessing import PolynomialFeatures

# This will be used to quantitatively evaluate our model
from sklearn.model_selection import cross_val_score

In [581]:
# this is given

def true_distribution(X):
    return np.cos(1.5 * np.pi * X)


np.random.seed(0)

sample_count = 50
noise_level = 0.1

random_noise = np.random.randn(sample_count) * noise_level    # creating noise
X = np.random.rand(sample_count)                              # creating random sample  
y = true_distribution(X) + random_noise                       # adding noise
X_true = np.linspace(0, 1, 100)                               # uniform distribution from 0-1. 

## Overfitting model parameters

**Question 5**

In this question, you will train two different prediction models on Galton's child-height dataset from lecture and explore different ways in how overfitting can appear.

**Part 1: Decision Tree Regressor**

* A decision tree regressor is trained similar to decision tree classifiers: the splits of the tree are created by minimizing the variance of the target values of the (training) data in the leaves given by making the split in question. 

* A decision tree regressor predicts the target value of a (new) observation based on the average target value of the training observations lying in the same leaf node. 

* One parameter of a decision tree regressor that affects model complexity is the *depth* of the tree. We will explore this parameter in this question.

* Create a function `tree_reg_perf` that takes in a dataframe like `galton` and outputs a dataframe where each row contains the *RSME* of a trained decision tree regressor on the training set and test set, indexed by the depth of the decision tree (depth=1,2,3,...,20).

*Note* (Optional question good for studying): How is this overfitting and why? What type of variance is causing it? What is the best choice of depth? Plot the dataframe above to help answer these questions.

**Part 2: k-Nearest Neighbor Regressor**

* A k-NN Regressor is predicts the target value of a (new) observation by computing the average value of the k-closest observations in the training set.

* One parameter of a k-NN regressor that affects model performance is the number of neighbors averaged over. We will explore this parameter in this question.

* Create a function `knn_reg_perf` that takes in a dataframe like `galton` and outputs a dataframe where each row contains the *RSME* of a trained k-NN regressor on the training set and test set, indexed by the number of neighbors (k=1,2,3,...,20).

*Note* (Optional question good for studying): How is this overfitting and why? What type of variance is causing it? What is the best choice for the number of neighbors? Plot the dataframe above to help answer these questions.


In [6]:
galton_fp = os.path.join('data', 'galton.csv')
galton = pd.read_csv(galton_fp)

## Titanic: predicting survival

**Question 6**

Predicting survival on the titanic is a common first assignment when learning classification tasks. There is *a lot* of material out there on analyzing the Titanic data. While not necessary for this question, you are encouraged to look at examples out on the web (e.g. [on kaggle](https://www.kaggle.com/c/titanic)).

Create a function `titanic_model` that takes in training data (a dataframe) and returns a pipeline object fit to the training data. You have freedom to build your own model, but it should satisfy the following requirements:

* The model is built on the target column `Survived` (which is not included in the training data).
* The model uses features derived from *all* other columns in training.
* You have one feature derived from the 'title' in the `Name` field ('Mr', 'Miss', 'Master', etc).
* You have one feature that scales the age of a passenger to standard unit among their `Pclass`.
* You can use any classification algorithm, as long as your performance is above the baseline accuracy 0.78.
* If your model (under retraining) can consistently score above 0.83, you can get 5 points extra-credit.
    - Your model will be trained using the training data given in the HW; it will be evaluated using held out test data.

*Note*: The function that is turned in should have the model parameters hard-coded in it. The pipeline object doesn't have to include the parameter selection process.

In [15]:
titanic_fp = os.path.join('data', 'titanic.csv')
titanic = pd.read_csv(titanic_fp)

## Document classification

**Question 7**

One of the problems that we often solve in NLP is a classification problem: given a set of documents and their labels (spam/not spam, positive/negative etc), one needs to build a model that can predicts the correct label on a new document. In order to apply classifiers (like random forest or Naive Bayes) we need to transform the text into a (set of) feature vector(s). We already know how it can be done: using a `bag of words`. In short, you take the available words in a text and keep count when they appear. 

### Bigrams and Trigrams

One idea that can help us generate text even better is to use grouped words. This changes allows the bag-of-words model to get more details about the document. This approach is called **n-grams**. 

**Example**

Let's look at the all bigrams for the following sentence: 
`The White house is open today`.

All the bigrams are:

* the White
* White house
* house is
* is open
* open today

As you can see, the bag-of-bigrams describes the meaning much better than the bag-of-words approach ('white', 'house', 'open', 'today'). 

In this exercise we would like you develop a general approach to a text classification problem and then use different settings (text preprocessing, different classification algorithms etc) to improve the learning rates.


* You are given a data file (`reviews.json`) that you will use to classify Amazon reviews. First you need to extract the text and the rating for each review; Also you need to clean each review before building a model: (convert everything to a lower-case and replace everything but letters, numbers and spaces with a space. In order to do that write a function `json_reader` that takes a file and the number of lines you want to read from the file. It returns two lists: one with cleaned reviews and another one with corresponding ratings (labels). 

* Now write a function `create_classifier_multi` that takes reviews and labels and returns a fit pipeline. You should reserve 20% of you training data for testing. Then set up a classifier using a Pipeline object such that it gives you the highest possible accuracy. Here is a trick: the accuracy is not known beforehand and you should try different classifiers and change their parameters, different pre-processing steps (for example: ngrams, stop words etc) in order to maximize the classifier's score. What is the highest value that you got? 

Do not be discouraged if you can't get a very high accuracy. Think why might it be the case? What if we do not use any algorithms and just assign labels to a new review randomly. What is the chance that you guessed the label correctly? Your accuracy does not seem that bad anymore, right? (If it is below a random assignment then change your code NOW).

In general, a multi-class classification problem is not an easy task. 

* Next you will convert the multi-class classification to the binary classification problem. In order to do that you need to write a method `to_binary` that takes in a list of labels and replaces all 1, 2, 3 reviews with a 0 and 4, 5 reviews with the 1. Build the model again by using different classifiers and pre-processing steps, by writing `create_classifier_binary` method and returning a pipeline that maximized the accuracy. Do you see the improvement? 

In [577]:
fp = os.path.join('data', 'reviews.json')



***Congratulations, you're done with the homework***

Now, run your doctests and upload hw08.py to GradeScope.